In [1]:
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import torch

# get our predefined functions (for data processing, etc.)
import QCircNet.utils as ut

# get our predefined quantum circuits and neural networks
import QCircNet.circuits.CustomCircuits as cn
import QCircNet.BinaryQuantumNN as qnn


EXPERIMENT_ID = 1
HYBRID = True

X_PATH = "data/X_apd_class6_7.npy"
Y_PATH = "data/y_apd_class6_7.npy"
BIN_ENCODING = {6: 0, 7: 1}


N_QUBITS = 4
FEAT_PER_QUBIT = 4

SCALER = MinMaxScaler()

SUBSET = None

EPOCHS = 16
WEIGHT_PATH = f"results/weights/bin/"
MODEL_PATH = f"results/models/bin/"
RESULTS_PATH = f"results/bin_model_results_expID_{EXPERIMENT_ID}.csv"
RESULTS_DF = pd.DataFrame()

VALIDATION = True

# for printing colored text output during training
RED_TEXT = "\033[91m"
GREEN_TEXT = "\033[92m"
RESET_COLOR = "\033[0m" 


In [2]:
# load the data
X_train, X_val, X_test, y_train, y_val, y_test, _ = ut.load_and_prepare_data(X_PATH, Y_PATH, scaler=SCALER, bin_encoding=BIN_ENCODING, subset=SUBSET)

# check shapes
print("train:\t", X_train.shape, y_train.shape)
print("val:\t", X_val.shape, y_val.shape)
print("test:\t", X_test.shape, y_test.shape)

train:	 torch.Size([9746, 16]) torch.Size([9746])
val:	 torch.Size([1219, 16]) torch.Size([1219])
test:	 torch.Size([1218, 16]) torch.Size([1218])


In [3]:
cn.get_custom_circuits()

[QCircNet.circuits.CustomCircuits.Anusha2024Circuit,
 QCircNet.circuits.CustomCircuits.DoubleEntanglementVLCircuit,
 QCircNet.circuits.CustomCircuits.Ranga2024Circuit,
 QCircNet.circuits.CustomCircuits.Senokosov2024Circuit]

In [4]:
circuits = [
    cn.Ranga2024Circuit,
    cn.Senokosov2024Circuit,
    cn.Anusha2024Circuit,
    cn.DoubleEntanglementVLCircuit,
]

runs = list(range(4, 11)) # runs are also used as seeds (reproducibility)
learning_rates = [0.01, 0.001]

In [ ]:
### RUN EXPERIMENT
id = 0
for seed, circuit, lr in itertools.product(runs, circuits, learning_rates):
    print(f"""{RED_TEXT}RUN {id+1}:{RESET_COLOR}\n {circuit.__name__} \n lr: {lr}""")

    # init the model
    model = qnn.BinQuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=seed, hybrid=HYBRID)

    path_identifier = f"{circuit.__name__}_expID_{EXPERIMENT_ID}_dfID_{id}"

    # train the model (predifined optimizer, etc.)
    trained_model, loss_history, avg_epoch_time = ut.train_model(
        model, X_train, y_train,
        epochs=EPOCHS, batch_size=32, lr=lr, binary=True,
    )
    
    # save trained model
    model_path = MODEL_PATH+path_identifier+".pth"
    torch.save(trained_model, model_path)

    # evaluate
    cross_entropy, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)

    results_dict = {
        "id": id,
        "f1_score": f1, "precision": precision, "recall": recall, "accuracy": accuracy, "cross_entropy": cross_entropy,
        "seed": seed,
        "circuit": circuit.__name__,
        "learning_rate": lr,
        "loss_history": loss_history,
        "avg_epoch_time_seconds": avg_epoch_time,
        "network_str": str(model),
        "model_path": model_path,
        "epochs": EPOCHS,
        "n_qubits": N_QUBITS,
        "features_per_qubit": FEAT_PER_QUBIT,
        "scaler" : str(SCALER).replace("()", "")
    }
    print(f"\n{GREEN_TEXT}RESULTS:\n F1-Score:{f1}, Precision:{precision}, Recall:{recall}, Accuracy:{accuracy}, Loss:{cross_entropy} {RESET_COLOR} \n\n")
    
    # append the results_dict to the RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([results_dict])], ignore_index=True)
    RESULTS_DF.to_csv(path_or_buf=RESULTS_PATH) # save for each run (in case of a crash)

    id += 1

RUN 1:
 Ranga2024Circuit 
 lr: 0.01


Training progress:  15%|█▍        | 45/304 [00:23<02:14,  1.93batch/s]

In [ ]:
RESULTS_DF

,id,f1_score,precision,recall,accuracy,cross_entropy,seed,circuit,learning_rate,loss_history,val_losses,val_f1s,avg_epoch_time_seconds,network_str,model_path,weight_path,epochs,n_qubits,features_per_qubit,scaler
0,0,0.678959,0.513957,1.000000,0.513957,0.631876,1,Ranga2024Circuit,0.010,"[0.65782711890183, 0.6308786188693423]","[0.6315123438835144, 0.6312209963798523]","[0.6793066088840737, 0.6793066088840737]",93.160429,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Ranga2024Circuit_expID_1_df...,results/weights/bin/,2,4,4,MinMaxScaler
1,1,0.678959,0.513957,1.000000,0.513957,0.686326,1,Ranga2024Circuit,0.001,"[0.7104883829229757, 0.6904856570457157]","[0.6969649791717529, 0.6852450370788574]","[0.6793066088840737, 0.6793066088840737]",92.678884,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Ranga2024Circuit_expID_1_df...,results/weights/bin/,2,4,4,MinMaxScaler
2,2,0.678959,0.513957,1.000000,0.513957,0.673887,1,Senokosov2024Circuit,0.010,"[0.6865284499761305, 0.6750583191843409]","[0.6780183911323547, 0.6731646656990051]","[0.6793066088840737, 0.6793066088840737]",105.048479,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Senokosov2024Circuit_expID_...,results/weights/bin/,2,4,4,MinMaxScaler
3,3,0.678959,0.513957,1.000000,0.513957,0.688581,1,Senokosov2024Circuit,0.001,"[0.7233674843844614, 0.6976102016081935]","[0.7090455293655396, 0.6883103251457214]","[0.6793066088840737, 0.6793066088840737]",101.043025,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Senokosov2024Circuit_expID_...,results/weights/bin/,2,4,4,MinMaxScaler
4,4,0.754217,0.605416,1.000000,0.665025,0.645970,1,Anusha2024Circuit,0.010,"[0.6504390418137375, 0.643677688350803]","[0.6456533074378967, 0.6453799605369568]","[0.7768800497203232, 0.7587878787878788]",30.826045,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Anusha2024Circuit_expID_1_d...,results/weights/bin/,2,4,4,MinMaxScaler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,0.856043,0.826152,0.888179,0.846470,0.572392,10,Senokosov2024Circuit,0.001,"[0.6714938463349092, 0.6001782715320587]","[0.629224956035614, 0.5733093023300171]","[0.7639257294429708, 0.8482490272373541]",101.438121,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Senokosov2024Circuit_expID_...,results/weights/bin/,2,4,4,MinMaxScaler
76,76,0.811166,0.834459,0.789137,0.811166,0.511324,10,Anusha2024Circuit,0.010,"[0.642745738358874, 0.52155496464356]","[0.5727839469909668, 0.5124857425689697]","[0.769927536231884, 0.8023064250411862]",30.867648,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Anusha2024Circuit_expID_1_d...,results/weights/bin/,2,4,4,MinMaxScaler
77,77,0.643397,0.505947,0.883387,0.496716,0.679080,10,Anusha2024Circuit,0.001,"[0.7111103883699367, 0.6890819235459754]","[0.7003628611564636, 0.6839656233787537]","[0.6387921022067363, 0.6339181286549708]",31.034930,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Anusha2024Circuit_expID_1_d...,results/weights/bin/,2,4,4,MinMaxScaler
78,78,0.804348,0.843860,0.768371,0.807882,0.557798,10,DoubleEntanglementVLCircuit,0.010,"[0.6087245401975355, 0.5588885808461591]","[0.5727593898773193, 0.5565664172172546]","[0.8693659281894576, 0.806070826306914]",53.473401,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/DoubleEntanglementVLCircuit...,results/weights/bin/,2,4,4,MinMaxScaler
